In [ ]:
import json
import os
import pandas as pd
import joblib  # or pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Load the model
def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "random_forest_model.pkl"))
    return model

# Handle input data
def input_fn(request_body, request_content_type):
    if request_content_type == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        # Load Excel file into a DataFrame
        df = pd.read_excel(request_body)
        return df
    else:
        raise ValueError("Unsupported content type: {}".format(request_content_type))

# Make predictions and calculate performance metrics
def predict_fn(input_data, model):
    # Assuming the last column is the target variable
    X = input_data.drop(columns=['score'])  # Features
    y_true = input_data['score']  # True labels

    # Predict using the model
    y_pred = model.predict(X)

    # Calculate performance metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    precision_micro = precision_score(y_true, y_pred, average='micro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    f1_micro = f1_score(y_true, y_pred, average='micro')
    conf_matrix = confusion_matrix(y_true, y_pred).tolist()  # Convert to list for JSON serialization

    # Return metrics rounded to 4 decimal places
    result = {
        "accuracy": round(accuracy, 4),
        "precision_macro": round(precision_macro, 4),
        "recall_macro": round(recall_macro, 4),
        "f1_macro": round(f1_macro, 4),
        "precision_micro": round(precision_micro, 4),
        "recall_micro": round(recall_micro, 4),
        "f1_micro": round(f1_micro, 4),
        "confusion_matrix": conf_matrix
    }
    return result

# Format the output
def output_fn(prediction, content_type):
    return json.dumps(prediction)
